In [1]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.2 MB/s eta 0:00:00


In [2]:
!huggingface-cli download TheBloke/finance-chat-GGUF finance-chat.Q5_K_M.gguf --local-dir . --local-dir-use-symlinks False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/commands/download.py:139: FutureWarning: Ignoring --local-dir-use-symlinks. Downloading to a local directory does not use symlinks anymore.
  warnings.warn(
finance-chat.Q5_K_M.gguf: 100%|████████████| 4.78G/4.78G [00:57<00:00, 83.4MB/s]
Download complete. Moving file to finance-chat.Q5_K_M.gguf
finance-chat.Q5_K_M.gguf


In [3]:
!pip install requests

import requests
import json
import zlib

YAHOO_TICKERS_URL = "https://yahoo-finance15.p.rapidapi.com/api/v2/markets/tickers?page=1&type=STOCKS"
YAHOO_TICKERS_HEADERS = {
    "x-rapidapi-host": "yahoo-finance15.p.rapidapi.com",
    "x-rapidapi-key": "e5dfb69ac4msh7dcab92ff8a5633p1e73d2jsncf8f43e4a966",
    "Accept": "application/json",
    "Accept-Encoding": "identity"
}
ESG_API_URL_TEMPLATE = "https://yahoo-finance127.p.rapidapi.com/esg-scores/{symbol}"
ESG_API_HEADERS = {
    "x-rapidapi-host": "yahoo-finance127.p.rapidapi.com",
    "x-rapidapi-key": "e5dfb69ac4msh7dcab92ff8a5633p1e73d2jsncf8f43e4a966",
    "Accept": "application/json",
    "Accept-Encoding": "identity"
}

def test_ticker_api():
    print("Testing Yahoo Finance Ticker API...")
    try:
        headers = YAHOO_TICKERS_HEADERS.copy()
        headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        
        response = requests.get(YAHOO_TICKERS_URL, headers=headers, timeout=10)
        print(f"Ticker API Status Code: {response.status_code}")
        print("Ticker API Response Headers:")
        print(response.headers)
        try:
            data = response.json()
            print("Ticker API JSON Response:")
            print(json.dumps(data, indent=2))
        except json.JSONDecodeError as e:
            print("JSON decode error:", e)
            print("Response content (first 200 characters):")
            print(response.content[:200])
    except Exception as e:
        print("Exception during ticker API test:", str(e))

def test_esg_api(symbol="AAPL"):
    print(f"\nTesting ESG API for symbol: {symbol}...")
    try:
        url = ESG_API_URL_TEMPLATE.format(symbol=symbol)
        headers = ESG_API_HEADERS.copy()
        headers["User-Agent"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
        
        response = requests.get(url, headers=headers, timeout=10)
        print(f"ESG API Status Code for {symbol}: {response.status_code}")
        print("ESG API Response Headers:")
        print(response.headers)
        try:
            data = response.json()
            print(f"ESG API JSON Response for {symbol}:")
            print(json.dumps(data, indent=2))
        except json.JSONDecodeError as e:
            print("JSON decode error for ESG API:", e)
            print("Response content (first 200 characters):")
            print(response.content[:200])
    except Exception as e:
        print(f"Exception during ESG API test for {symbol}:", str(e))

test_ticker_api()
test_esg_api("AAPL")


Testing Yahoo Finance Ticker API...
Ticker API Status Code: 200
Ticker API Response Headers:
{'Date': 'Sat, 08 Mar 2025 10:57:52 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'expires': '-1', 'Cache-Control': 'private, must-revalidate', 'NEL': '{"success_fraction":0,"report_to":"cf-nel","max_age":604800}', 'x-xss-protection': '1; mode=block', 'vary': 'Origin, accept-encoding', 'alt-svc': 'h3=":443"; ma=86400', 'x-content-type-options': 'nosniff', 'CF-RAY': '91d1d06c7fc739a4-IAD', 'cf-cache-status': 'DYNAMIC', 'server-timing': 'cfL4;desc="?proto=TCP&rtt=2361&min_rtt=1766&rtt_var=1022&sent=5&recv=6&lost=0&retrans=0&sent_bytes=2995&recv_bytes=1277&delivery_rate=1653454&cwnd=252&unsent_bytes=0&cid=d87330b9857433c1&ts=369&x=0"', 'x-frame-options': 'SAMEORIGIN', 'Report-To': '{"endpoints":[{"url":"https:\\/\\/a.nel.cloudflare.com\\/report\\/v4?s=KmPH22STAjfgvvIfyJTpk8R1I2R4%2BaBbELvDC2qHqwvlR9K%2BX1sItDhnyB4TZr9CL7nVgYVKht3YMHGXOrtTF2gH

In [9]:
!pip install llama-cpp-python


  Using cached llama_cpp_python-0.3.7.tar.gz (66.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl.metadata (20 kB)
Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.7-cp310-cp310-linux_x86_64.whl size=4601123 sha256=5ef59c9115a9a80f9493dd331e9369db3a87b3a35a60061f01b9f3c077d9fcce
  Stored in directory: /root/.cache/pip/wheels/5c/8f/58/a39eb13258f3bbf64bb36ed76d31979579a6f175be38de06b7
Successfully built llama-cpp-python


In [10]:

import requests
from llama_cpp import Llama
import logging
import time
import json

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("finance_api.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)





In [11]:

YAHOO_TICKERS_URL = "https://yahoo-finance15.p.rapidapi.com/api/v2/markets/tickers"
YAHOO_TICKERS_HEADERS = {
    "x-rapidapi-host": "yahoo-finance15.p.rapidapi.com",
    "x-rapidapi-key": "e5dfb69ac4msh7dcab92ff8a5633p1e73d2jsncf8f43e4a966",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

ESG_API_URL_TEMPLATE = "https://yahoo-finance127.p.rapidapi.com/esg-scores/{symbol}"
ESG_API_HEADERS = {
    "x-rapidapi-host": "yahoo-finance127.p.rapidapi.com",
    "x-rapidapi-key": "e5dfb69ac4msh7dcab92ff8a5633p1e73d2jsncf8f43e4a966",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

In [22]:
def get_all_ticker_pages(max_pages=20, symbol_to_find=None):

    all_tickers = []
    found_symbol = False
    
    for page in range(1, max_pages + 1):
        logger.info(f"Fetching ticker page {page}/{max_pages}")
        
        try:
            params = {
                "page": page,
                "type": "STOCKS"
            }
            
            # Add backoff retry logic
            max_retries = 3
            retry_delay = 2  # seconds
            
            for attempt in range(max_retries):
                try:
                    response = requests.get(
                        YAHOO_TICKERS_URL, 
                        headers=YAHOO_TICKERS_HEADERS, 
                        params=params,
                        timeout=10
                    )
                    
                    # Log response details
                    logger.info(f"Page {page} status code: {response.status_code}")
                    logger.info(f"Page {page} headers: {dict(response.headers)}")
                    
                    if response.status_code == 200:
                        data = response.json()
                        logger.info(f"Page {page} response structure: {list(data.keys())}")
                        
                        # Check if body is in the response
                        if "body" in data and isinstance(data["body"], list):
                            page_tickers = data["body"]
                            logger.info(f"Found {len(page_tickers)} tickers in page {page}")
                            
                            # Add tickers from this page to our collection
                            all_tickers.extend(page_tickers)
                            
                            # If looking for a specific symbol, check if it's in this page
                            if symbol_to_find:
                                for ticker in page_tickers:
                                    if ticker.get("symbol") == symbol_to_find:
                                        logger.info(f"Found target symbol {symbol_to_find} on page {page}")
                                        found_symbol = True
                                        break
                        else:
                            logger.warning(f"No 'body' list found in page {page} response")
                            # Log the actual response structure for debugging
                            logger.warning(f"Response content: {response.text[:500]}...")
                        
                        # Success, break the retry loop
                        break
                    
                    elif response.status_code == 429:  # Rate limit exceeded
                        retry_after = int(response.headers.get('Retry-After', retry_delay * (attempt + 1)))
                        logger.warning(f"Rate limit hit on attempt {attempt+1}, waiting {retry_after} seconds")
                        time.sleep(retry_after)
                    
                    else:
                        logger.error(f"Error fetching page {page}: Status {response.status_code}")
                        logger.error(f"Response: {response.text[:500]}...")
                        break  # Non-retryable error
                
                except requests.exceptions.RequestException as e:
                    logger.error(f"Request exception on page {page}, attempt {attempt+1}: {str(e)}")
                    if attempt < max_retries - 1:
                        time.sleep(retry_delay * (attempt + 1))
                    else:
                        logger.error(f"Max retries reached for page {page}")
            
            # If we found our target symbol or hit an error, stop fetching pages
            if found_symbol:
                break
                
        except Exception as e:
            logger.exception(f"Unexpected error fetching page {page}: {str(e)}")
    
    logger.info(f"Total tickers collected: {len(all_tickers)}")
    return all_tickers

In [13]:
def search_ticker_by_symbol(symbol, tickers=None, max_pages=3):

    logger.info(f"Searching for ticker symbol: {symbol}")
    
    # Use provided tickers if available, otherwise fetch them
    if tickers is None:
        tickers = get_all_ticker_pages(max_pages=max_pages, symbol_to_find=symbol)
    
    # Search for the symbol in our ticker data
    for ticker in tickers:
        if ticker.get("symbol") == symbol:
            logger.info(f"Found ticker data for {symbol}")
            logger.debug(f"Ticker data: {ticker}")
            return ticker
    
    logger.warning(f"Ticker {symbol} not found in {len(tickers)} results")
    return None

def get_esg_data(symbol):
    """Fetch ESG score data for a given symbol from Yahoo Finance ESG API."""
    logger.info(f"Fetching ESG data for symbol: {symbol}")
    
    try:
        url = ESG_API_URL_TEMPLATE.format(symbol=symbol)
        
        # Add retry logic
        max_retries = 3
        retry_delay = 2  # seconds
        
        for attempt in range(max_retries):
            try:
                response = requests.get(
                    url, 
                    headers=ESG_API_HEADERS,
                    timeout=10
                )
                
                logger.info(f"ESG API status code for {symbol}: {response.status_code}")
                
                if response.status_code == 200:
                    data = response.json()
                    logger.info(f"ESG data keys for {symbol}: {list(data.keys())}")
                    return data
                
                elif response.status_code == 429:  # Rate limit exceeded
                    retry_after = int(response.headers.get('Retry-After', retry_delay * (attempt + 1)))
                    logger.warning(f"Rate limit hit on ESG for {symbol}, waiting {retry_after} seconds")
                    time.sleep(retry_after)
                
                else:
                    logger.error(f"ESG API error for {symbol}: Status {response.status_code}")
                    logger.error(f"Response: {response.text[:500]}...")
                    return {}
                    
            except requests.exceptions.RequestException as e:
                logger.error(f"Request exception on ESG for {symbol}, attempt {attempt+1}: {str(e)}")
                if attempt < max_retries - 1:
                    time.sleep(retry_delay * (attempt + 1))
                else:
                    logger.error(f"Max retries reached for ESG data on {symbol}")
                    return {}
        
    except Exception as e:
        logger.exception(f"Unexpected error fetching ESG data for {symbol}: {str(e)}")
        return {}


In [14]:
def check_model_file(model_path):
    """Check if the model file exists and log its status."""
    import os
    if os.path.exists(model_path):
        model_size = os.path.getsize(model_path) / (1024 * 1024)  # Size in MB
        logger.info(f"Model file found at {model_path} ({model_size:.2f} MB)")
        return True
    else:
        logger.error(f"Model file not found at {model_path}")
        return False

def initialize_llama(model_path):
    """Initialize the Llama model with GPU support and fallback to CPU."""
    logger.info("Initializing Llama model with GPU support...")
    
    if not check_model_file(model_path):
        logger.error("Cannot initialize model: file not found")
        return None
    
    try:
        # First try to initialize with GPU support
        model = Llama(
            model_path=model_path,
            n_ctx=4096,
            n_threads=4,
            n_gpu_layers=-1  # Use all layers on GPU
        )
        logger.info("Llama model initialized successfully with GPU support")
        return model
    except Exception as e:
        logger.exception(f"Error initializing Llama model with GPU: {str(e)}")
        logger.info("Falling back to CPU initialization...")
        try:
            # Fallback to CPU
            model = Llama(
                model_path=model_path,
                n_ctx=4096,
                n_threads=4
            )
            logger.info("Llama model initialized successfully on CPU")
            return model
        except Exception as e2:
            logger.exception(f"Error initializing Llama model on CPU: {str(e2)}")
            return None

In [15]:


def chat_response(user_message, model_path="/kaggle/working/finance-chat.Q5_K_M.gguf"):
    """
    Generate a response to a user's financial query with detailed logging.
    
    Args:
        user_message: The user's query
        model_path: Path to the GGUF model file
        
    Returns:
        Generated response string
    """
    logger.info(f"Processing user query: {user_message}")
    
    # Check if query is specifically looking for a symbol
    import re
    symbol_match = re.search(r'\b([A-Z]{1,5})\b', user_message)
    target_symbol = None
    
    if symbol_match:
        potential_symbol = symbol_match.group(1)
        if len(potential_symbol) >= 2:  # Most stock symbols are at least 2 characters
            target_symbol = potential_symbol
            logger.info(f"Detected potential stock symbol in query: {target_symbol}")
    
    # Fetch all ticker data or search for specific symbol
    if target_symbol:
        # First, try to directly get the specific ticker
        ticker_data = search_ticker_by_symbol(target_symbol, max_pages=20)
        if ticker_data:
            tickers = [ticker_data]
        else:
            # If not found, get general top tickers
            tickers = get_all_ticker_pages(max_pages=2)[:3]
    else:
        # No specific symbol, get top tickers
        tickers = get_all_ticker_pages(max_pages=2)[:3]
    
    # Prepare ticker summary for prompt
    ticker_summary = ""
    for ticker in tickers:
        symbol = ticker.get("symbol", "N/A")
        price = ticker.get("lastsale", "N/A")
        change_pct = ticker.get("pctchange", "N/A")
        
        # Log ticker details
        logger.info(f"Including ticker in summary: {symbol} @ {price} ({change_pct})")
        
        # Only fetch ESG data for the target symbol or for display tickers if no target
        if symbol == target_symbol or not target_symbol:
            esg_data = get_esg_data(symbol)
            if esg_data and "totalEsg" in esg_data:
                esg_score = esg_data["totalEsg"].get("fmt", "N/A")
                env_score = esg_data.get("environmentScore", {}).get("fmt", "N/A")
                social_score = esg_data.get("socialScore", {}).get("fmt", "N/A")
                gov_score = esg_data.get("governanceScore", {}).get("fmt", "N/A")
                
                esg_text = (f"ESG Score: {esg_score}, Environmental: {env_score}, "
                           f"Social: {social_score}, Governance: {gov_score}")
                logger.info(f"ESG data for {symbol}: {esg_text}")
            else:
                esg_text = "No ESG data available"
                logger.warning(f"No ESG data found for {symbol}")
        else:
            esg_text = "ESG data not fetched"
        
        ticker_summary += f"{symbol}: Price {price} ({change_pct}); {esg_text}. "
    
    if not ticker_summary:
        ticker_summary = "No ticker information available."
        logger.warning("No ticker data could be retrieved")
    
    # Log the final ticker summary
    logger.info(f"Final ticker summary: {ticker_summary}")
    
    # Initialize the model with GPU support
    llama = initialize_llama(model_path)
    if not llama:
        return "I'm sorry, but I'm unable to process your request at the moment due to a technical issue with the language model."
    
    # Construct the enriched prompt
    enriched_prompt = (
        "Answer the following financial query by providing a detailed analysis, "
        "recommendations for greener and sustainable alternatives, and insights on sustainable finance. "
        "Focus on clear, actionable financial Q&A while incorporating the data provided below.\n\n"
        f"User Query: {user_message}\n\n"
        f"Ticker and ESG Information: {ticker_summary}\n\n"
        "Answer:"
    )
    
    logger.info("Sending prompt to Llama model")
    logger.debug(f"Full prompt: {enriched_prompt}")
    
    try:

        output = llama(
            prompt=enriched_prompt,
            max_tokens=800,
            temperature=0.7,
            top_p=0.95,
        )
        
        if isinstance(output, dict):
            logger.info("Llama returned dictionary output")
            reply = output.get("choices", [{}])[0].get("text", "")
        elif isinstance(output, str):
            logger.info("Llama returned string output")
            reply = output
        else:
            logger.warning(f"Unexpected output type from Llama: {type(output)}")
            if hasattr(output, "__getitem__"):
                try:
                    first_item = output[0]
                    if isinstance(first_item, dict) and "text" in first_item:
                        reply = first_item["text"]
                    else:
                        reply = str(first_item)
                except (IndexError, TypeError):
                    reply = str(output)
            else:
                reply = str(output)
        
        logger.info("Successfully generated response")
        logger.debug(f"Model response: {reply}")
        
        return reply.strip()
        
    except Exception as e:
        logger.exception(f"Error generating response with Llama: {str(e)}")
        return "I apologize, but I encountered an error while processing your request. Please try again with a different query."


In [23]:
# Add CUDA installation code if needed
def install_cuda_support():
    """Install CUDA-enabled version of llama-cpp-python."""
    try:
        logger.info("Installing CUDA-enabled llama-cpp-python...")
        import sys
        import subprocess
        
        # Uninstall current version
        subprocess.check_call([sys.executable, "-m", "pip", "uninstall", "-y", "llama-cpp-python"])
        
        # Install with CUDA support
        env = {"CMAKE_ARGS": "-DLLAMA_CUBLAS=on", "FORCE_CMAKE": "1"}
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "llama-cpp-python"],
            env=env
        )
        
        logger.info("Successfully installed CUDA-enabled llama-cpp-python")
        return True
    except Exception as e:
        logger.exception(f"Failed to install CUDA support: {str(e)}")
        return False

if __name__ == "__main__":
    
    user_query = "give me some sustainable finance strategies and some sustainable stocks"
    
    logger.info("Starting test run with query: " + user_query)
    
    try:
        reply = chat_response(user_query)
        print("\nChatbot Response:\n" + "-"*50)
        print(reply)
        print("-"*50)
        
        logger.info("Test completed successfully")
        
    except Exception as e:
        logger.exception(f"Test failed with exception: {str(e)}")

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /kaggle/working/finance-chat.Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_cou


Chatbot Response:
--------------------------------------------------
[/INST] Based on the user's request for sustainable finance strategies and sustainable stocks, I will provide a detailed analysis of the three stocks mentioned, ABBV, HD, and FMX, and recommend sustainable investing strategies.

Firstly, I will analyze the ESG scores of the three stocks. ABBV has an ESG score of 26.0, with the lowest score being 1.0. HD has an ESG score of 12.6, and FMX has no ESG data available. Based on these scores, ABBV and HD have better ESG scores than FMX.

Next, I will recommend sustainable investing strategies for ABBV, HD, and FMX. For ABBV, I recommend investing in renewable energy stocks, such as TAN, FAN, and DCP. For HD, I recommend investing in sustainable packaging companies, such as MPC, CPK and KDP. For FMX, I recommend investing in sustainable water companies, such as AWK and CWT.

In addition to investing in sustainable stocks, investors can also consider investing in environmenta